In [27]:
import datetime as dt
import json
import time
import random
from datetime import datetime
import traceback
from tqdm import tqdm

from bitshares import BitShares
from bitshares.account import Account
from bitshares.wallet import Wallet
from bitsharesbase.account import BrainKey
from bitshares.blockchain import Blockchain
from bitsharesbase.chains import known_chains

In [28]:
web_socket = "ws://testnet-seed-7.gravityprotocol.org:4624"
chain_id = "ab5071857c28ddbc872d0ca508725fa3006ea7bdfda10f707433021f570fc27e" 
known_chains = { "ZGV": {"chain_id": "ab5071857c28ddbc872d0ca508725fa3006ea7bdfda10f707433021f570fc27e",
                         "core_symbol": "ZGV",
                         "prefix": "ZGV"}}

testnet = BitShares(web_socket)
bc = Blockchain(testnet)
users = list(bc.get_all_accounts())

/Users/mac/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: generator 'Blockchain.get_all_accounts' raised StopIteration
  if __name__ == '__main__':


In [20]:
import csv

In [23]:
pre_emission_data = []
with open('testnet_first_emission/results/emission_2018-06-01_12-39-48.csv', newline='') as file:
    r = csv.reader(file, delimiter=';')
    for row in file:
        row = row.strip().split(';')
        pre_emission_data.append(row)

In [32]:
len(pre_emission_data)

1262

In [30]:
#Словари между id и именем аккаунта
user_ids = {}
user_names = {}
for u in users:
    try:
        acc = Account(u, bitshares_instance=testnet)
        user_ids[u] = acc['id']
        user_names[acc['id']] = u
    except:
        print(u)

In [31]:
#Поиск родительских нод по ключу registrar 
parents = []
kids = []
for user in tqdm(users):
    real_acc = Account(user, bitshares_instance=testnet)
    h = real_acc.history(first=None,last=0,limit=1000000,only_ops=[],exclude_ops=[])
    hista = list(h)
    #print(hista[-1]['op'][1])
    try:
        #print(hista[-1]['op'][1]['registrar'])
        #if hista[-1]['op'][1]['registrar'] not in parents:
        parents.append(hista[-1]['op'][1]['registrar'])
        kids.append(user)
        
    except:
        pass


 99%|█████████▉| 2377/2394 [11:39<00:05,  3.40it/s]/Users/mac/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: generator 'Account.history' raised StopIteration
  import sys
100%|██████████| 2394/2394 [11:41<00:00,  3.41it/s]


In [104]:
pars = [user_names[p] for p in set(parents)]

In [107]:
pars.remove('nathan')

In [108]:
# Словарь родитель и список его нод
parent_accounts = {}
for p in parents:
    if user_names[p] != 'nathan':
        name = user_names[p]
        parent_accounts[name] = []

In [109]:
for user in tqdm(users):
    real_acc = Account(user, bitshares_instance=testnet)
    h = real_acc.history(first=None,last=0,limit=1000000,only_ops=[],exclude_ops=[])
    hista = list(h)
    try:
        p = user_names[hista[-1]['op'][1]['registrar']]
        parent_accounts[p].append(user)
    except:
        pass

 99%|█████████▉| 2377/2394 [1:39:49<00:42,  2.52s/it]/Users/mac/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: generator 'Account.history' raised StopIteration
  after removing the cwd from sys.path.
100%|██████████| 2394/2394 [1:40:17<00:00,  2.51s/it]


In [110]:
parent_accounts.keys()

dict_keys(['g9320b3400s1180y4450', 'g9480a2230f7280f3790', 'g2390w9350c1220p0270', 'g8090b6110c6310f7000'])

In [111]:
parent_accounts

{'g2390w9350c1220p0270': ['g0039r8540v5126z0390',
  'g0068r2258v0455z8811',
  'g0068r2570v5802z0738',
  'g0106r2247v0024z4048',
  'g0109r8084v4921z9300',
  'g0221r3893v5766z6163',
  'g0282r1539v9120z4597',
  'g0303r3417v5008z0185',
  'g0323r2536v5603z5610',
  'g0323r5225v2310z6816',
  'g0387r9522v6778z5518',
  'g0404r2366v5487z8991',
  'g0404r5459v7530z0826',
  'g0406r7732v7754z3902',
  'g0416r5809v3279z3184',
  'g0432r2677v6546z2562',
  'g0437r2572v1415z4366',
  'g0461r8330v8638z5740',
  'g0475r0486v2117z8177',
  'g0477r0533v0992z3291',
  'g0497r3554v5851z2026',
  'g0514r3444v1263z2591',
  'g0565r7759v2073z3331',
  'g0628r0855v0983z1010',
  'g0641r9044v9195z0313',
  'g0649r5530v6367z3150',
  'g0678r8939v5309z5572',
  'g0688r5377v5660z9416',
  'g0691r6973v3219z2883',
  'g0709r7932v3150z6730',
  'g0719r1917v2394z2855',
  'g0721r8557v5371z8818',
  'g0787r3467v7776z4133',
  'g0791r2512v0927z4518',
  'g0794r8398v8841z4949',
  'g0805r9182v1306z2702',
  'g0817r7948v0614z5055',
  'g0842r6677v

In [117]:
#Родительская ноды и кол-во её акков
for p in set(parents):
    print(p, user_names[p], parents.count(p))

1.2.242 g9480a2230f7280f3790 1001
1.2.17 nathan 408
1.2.212 g8090b6110c6310f7000 99
1.2.259 g2390w9350c1220p0270 812
1.2.199 g9320b3400s1180y4450 12


In [112]:
groups = {}
groups['nathan'] = 'nathan'
groups['faucet'] = 'faucet'
groups['trees'] = [[key]+parent_accounts[key] for key in parent_accounts.keys() if key != parents]
kids = []
for key in parent_accounts:
    kids += parent_accounts[key]
groups['secluded'] = [u for u in users if u not in kids and u not in pars and u != 'nathan' and u != 'faucet']

In [119]:
for key in parent_accounts.keys():
    print(key, len(parent_accounts[key]))

g9320b3400s1180y4450 12
g9480a2230f7280f3790 1001
g2390w9350c1220p0270 509
g8090b6110c6310f7000 99


In [51]:
pre_emission_data[1]

['faucet',
 '54087161799593',
 '0',
 '0.00361538',
 '0.000362',
 '653667381638.169800']

In [81]:
emission_balances = {}
total_sum = 0
for u in pre_emission_data:
    total_sum += float(u[1])/100000
    emission_balances[u[0]] = float(u[1])/100000

In [87]:
#Полный суммарный баланс перед эмиссией
total_sum

999387634.6106406

In [90]:
total_activity = 0
activities = {}
for u in pre_emission_data:
    total_activity += float(u[3])
    activities [u[0]] = float(u[3])

In [91]:
total_activity

0.9999995999999993

In [92]:
total_gravity = 0
gravity = {}
for u in pre_emission_data:
    total_gravity += float(u[4])
    gravity[u[0]] = float(u[4])

In [93]:
total_gravity

0.09998599999999978

In [95]:
total_emission = 0
actual_emission = {}
for u in pre_emission_data:
    total_emission += float(u[5])/100000
    actual_emission[u[0]] = float(u[5])/100000

In [96]:
total_emission

13410962.58352171

In [63]:
#Те кто получил эмиссию
e_users = [u[0] for u in pre_emission_data]

In [120]:
#Балансы по группам
balance_groups = {}
balance_groups['faucet'] = emission_balances['faucet']
balance_groups['nathan'] = emission_balances['nathan']
balance_groups['secluded'] = 0
for key in emission_balances.keys():
    if key != 'nathan' and key != 'faucet' and key not in kids and key not in pars:
        balance_groups['secluded'] += emission_balances[key]

In [121]:
balance_groups['trees'] = {}
for key in emission_balances.keys():
    if key in pars:
        balance_groups['trees'][key] = emission_balances[key]
for key in balance_groups['trees'].keys():
    for k in parent_accounts[key]:
        try:
            balance_groups['trees'][key] += emission_balances[k]
        except:
            pass

In [132]:
balance_groups

{'faucet': 540871617.99593,
 'nathan': 149582582.92,
 'secluded': 250474320.33589986,
 'trees': {'g2390w9350c1220p0270': 9690.0,
  'g8090b6110c6310f7000': 1599077.71038,
  'g9320b3400s1180y4450': 101549.77652000007,
  'g9480a2230f7280f3790': 56748852.87191}}

In [123]:
index_balance = {}
index_balance['faucet'] = balance_groups['faucet']/total_sum
index_balance['nathan'] = balance_groups['nathan']/total_sum
index_balance['secluded'] = balance_groups['secluded']/total_sum

In [124]:
index_balance['trees'] = {}
for key in balance_groups['trees'].keys():
    index_balance['trees'][key] = balance_groups['trees'][key]/total_sum

In [125]:
index_balance

{'faucet': 0.5412030320013439,
 'nathan': 0.14967423824317883,
 'secluded': 0.25062779612385755,
 'trees': {'g2390w9350c1220p0270': 9.695937456515764e-06,
  'g8090b6110c6310f7000': 0.0016000575302324983,
  'g9320b3400s1180y4450': 0.00010161200019206127,
  'g9480a2230f7280f3790': 0.05678362519866401}}

In [126]:
activity_groups = {}
activity_groups['faucet'] = activities['faucet']
activity_groups['nathan'] = activities['nathan']
activity_groups['secluded'] = 0
for key in activities.keys():
    if key != 'nathan' and key != 'faucet' and key not in kids and key not in pars:
        activity_groups['secluded'] += activities[key]

In [127]:
activity_groups['trees'] = {}
for key in activities.keys():
    if key in pars:
        activity_groups['trees'][key] = activities[key]
for key in activity_groups['trees'].keys():
    for k in parent_accounts[key]:
        try:
            activity_groups['trees'][key] += activities[k]
        except:
            pass

In [128]:
activity_groups

{'faucet': 0.00361538,
 'nathan': 0.00270099,
 'secluded': 0.9798708699999992,
 'trees': {'g2390w9350c1220p0270': 0.00257328,
  'g8090b6110c6310f7000': 0.00135185,
  'g9320b3400s1180y4450': 0.00135185,
  'g9480a2230f7280f3790': 0.00982202}}

In [129]:
gravity_groups = {}
gravity_groups['faucet'] = gravity['faucet']
gravity_groups['nathan'] = gravity['nathan']
gravity_groups['secluded'] = 0
for key in gravity.keys():
    if key != 'nathan' and key != 'faucet' and key not in kids and key not in pars:
        gravity_groups['secluded'] += gravity[key]

In [130]:
gravity_groups['trees'] = {}
for key in gravity.keys():
    if key in pars:
        gravity_groups['trees'][key] = gravity[key]
for key in gravity_groups['trees'].keys():
    for k in parent_accounts[key]:
        try:
            gravity_groups['trees'][key] += gravity[k]
        except:
            pass

In [131]:
gravity_groups

{'faucet': 0.000362,
 'nathan': 0.00027,
 'secluded': 0.09797299999999978,
 'trees': {'g2390w9350c1220p0270': 0.000258,
  'g8090b6110c6310f7000': 0.000135,
  'g9320b3400s1180y4450': 0.000135,
  'g9480a2230f7280f3790': 0.000982}}

In [133]:
import csv

transactions = []
with open('pretty_logs.csv', newline='') as file:
    r = csv.reader(file, delimiter=';')
    for row in file:
        row = row.strip().split(';')
        transactions.append(row)

In [145]:
tree_tx = {}
tree_counter = {}
tree_volumes = {}
for key in parent_accounts.keys():
    tree_counter[key] = {}
    tree_volumes[key] = {}
    tree_tx[key] = []
for key in parent_accounts.keys():
    tree_counter[key]['from parent'] = 0
    tree_counter[key]['to parent'] = 0
    tree_counter[key]['kid to kid'] = 0
    tree_volumes[key]['from parent'] = 0
    tree_volumes[key]['to parent'] = 0
    tree_volumes[key]['kid to kid'] = 0
for tx in transactions:
    if tx[1] in pars and tx[2] in parent_accounts[tx[1]]:
        tree_tx[tx[1]].append(tx)
        tree_counter[tx[1]]['from parent'] += 1
        tree_volumes[tx[1]]['from parent'] += float(tx[3])
    if tx[2] in pars and tx[1] in parent_accounts[tx[2]]:
        tree_tx[tx[2]].append(tx)
        tree_counter[tx[2]]['to parent'] += 1
        tree_volumes[tx[2]]['to parent'] += float(tx[3])
    if tx[1] in kids and tx[2] in kids:
        for key in parent_accounts.keys():
            if tx[1] in parent_accounts[key] and tx[2] in parent_accounts[key]:
                tree_counter[key]['kid to kid'] += 1
                tree_volumes[key]['kid to kid'] += float(tx[3])

In [146]:
tree_counter

{'g2390w9350c1220p0270': {'from parent': 137,
  'kid to kid': 62092,
  'to parent': 128},
 'g8090b6110c6310f7000': {'from parent': 0, 'kid to kid': 0, 'to parent': 0},
 'g9320b3400s1180y4450': {'from parent': 24, 'kid to kid': 0, 'to parent': 0},
 'g9480a2230f7280f3790': {'from parent': 0, 'kid to kid': 0, 'to parent': 0}}

In [147]:
tree_volumes

{'g2390w9350c1220p0270': {'from parent': 105846855.0,
  'kid to kid': 46527291600.0,
  'to parent': 98371332.0},
 'g8090b6110c6310f7000': {'from parent': 0, 'kid to kid': 0, 'to parent': 0},
 'g9320b3400s1180y4450': {'from parent': 7001.0,
  'kid to kid': 0,
  'to parent': 0},
 'g9480a2230f7280f3790': {'from parent': 0, 'kid to kid': 0, 'to parent': 0}}

In [148]:
send_count = {}
receive_count = {}
send_volume = {}
receive_volume = {}
users = []

for tx in transactions:
    users.append(tx[1])
    users.append(tx[2])
users = set(users)

for u in users:
    send_count[u] = 0
    receive_count[u] = 0
    send_volume[u] = 0
    receive_volume[u] = 0

for tx in transactions:
    send_count[tx[1]] += 1
    receive_count[tx[2]] += 1
    send_volume[tx[1]] += float(tx[3]) 
    receive_volume[tx[2]] = float(tx[3])

In [149]:
sorted(send_count.items(), key=lambda x: x[1])[::-1]

[('g9480a2230f7280f3790', 107508),
 ('g3320r2802v9614z0736', 448),
 ('g5713r3823v9460z9289', 445),
 ('g7994r6747v1982z2731', 442),
 ('g2434r7947v5691z9463', 440),
 ('g8844r2937v7237z2881', 440),
 ('g5631r3575v4958z4393', 437),
 ('g4690r2399v7897z7827', 434),
 ('g2433r3358v5349z0109', 433),
 ('g9471r9666v7658z3920', 433),
 ('g6994r7613v4152z6566', 432),
 ('g5359r6617v8567z4963', 432),
 ('g0038r9894v0515z6306', 430),
 ('g7375r0132v2222z2209', 427),
 ('g7273r3106v2775z4066', 426),
 ('g6603r4182v4767z7852', 425),
 ('g4248r2689v0361z7550', 425),
 ('g8478r5578v5201z4888', 422),
 ('g2142r0760v6688z7003', 421),
 ('g7517r2836v8057z1209', 420),
 ('g7698r9605v3288z9411', 420),
 ('g0931r4288v9712z9223', 420),
 ('g8949r4079v9915z0232', 419),
 ('g4040r5021v2359z1661', 419),
 ('g1278r7145v9526z1646', 418),
 ('g8722r9550v5474z3435', 417),
 ('g5381r9509v8045z1084', 416),
 ('g6588r5017v3151z1566', 416),
 ('g7585r4787v9721z8486', 416),
 ('g5818r7818v7016z4691', 415),
 ('g4006r0570v1443z6709', 415),
 ('g0

In [150]:
sorted(receive_count.items(), key=lambda x: x[1])[::-1]

[('g1470c5390q8650k4640', 21515),
 ('g0080c2500c2040j4800', 21510),
 ('g7030o5660s1500g0480', 21503),
 ('g4320a3300y6850c5380', 21497),
 ('g9200w9590s3850z2720', 21490),
 ('g3320r2802v9614z0736', 448),
 ('g5713r3823v9460z9289', 445),
 ('g7994r6747v1982z2731', 442),
 ('g2434r7947v5691z9463', 440),
 ('g8844r2937v7237z2881', 440),
 ('g5631r3575v4958z4393', 437),
 ('g4690r2399v7897z7827', 434),
 ('g2433r3358v5349z0109', 433),
 ('g9471r9666v7658z3920', 433),
 ('g6994r7613v4152z6566', 432),
 ('g5359r6617v8567z4963', 432),
 ('g0038r9894v0515z6306', 430),
 ('g7375r0132v2222z2209', 427),
 ('g7273r3106v2775z4066', 426),
 ('g6603r4182v4767z7852', 425),
 ('g4248r2689v0361z7550', 425),
 ('g8478r5578v5201z4888', 422),
 ('g2142r0760v6688z7003', 421),
 ('g7517r2836v8057z1209', 420),
 ('g7698r9605v3288z9411', 420),
 ('g0931r4288v9712z9223', 420),
 ('g8949r4079v9915z0232', 419),
 ('g4040r5021v2359z1661', 419),
 ('g1278r7145v9526z1646', 418),
 ('g8722r9550v5474z3435', 417),
 ('g5381r9509v8045z1084', 416)

In [151]:
sorted(send_volume.items(), key=lambda x: x[1])[::-1]

[('nathan', 915642090.0),
 ('g2434r7947v5691z9463', 379563676.0),
 ('g5713r3823v9460z9289', 379471110.0),
 ('g3320r2802v9614z0736', 377938569.0),
 ('g8844r2937v7237z2881', 376954277.0),
 ('g7994r6747v1982z2731', 376792606.0),
 ('g2433r3358v5349z0109', 373876806.0),
 ('g5631r3575v4958z4393', 370723469.0),
 ('g4690r2399v7897z7827', 368348907.0),
 ('g0038r9894v0515z6306', 367940126.0),
 ('g7375r0132v2222z2209', 367209597.0),
 ('g5359r6617v8567z4963', 366534188.0),
 ('g6994r7613v4152z6566', 365999165.0),
 ('g7517r2836v8057z1209', 365774155.0),
 ('g7273r3106v2775z4066', 365756249.0),
 ('g9471r9666v7658z3920', 365285825.0),
 ('g4040r5021v2359z1661', 361106355.0),
 ('g8478r5578v5201z4888', 361058675.0),
 ('g4248r2689v0361z7550', 360796566.0),
 ('g2142r0760v6688z7003', 359713317.0),
 ('g6603r4182v4767z7852', 359505403.0),
 ('faucet', 359123000.0),
 ('g1278r7145v9526z1646', 359005144.0),
 ('g6588r5017v3151z1566', 358699317.0),
 ('g7585r4787v9721z8486', 358264767.0),
 ('g8722r9550v5474z3435', 35

In [152]:
sorted(receive_volume.items(), key=lambda x: x[1])[::-1]

[('g0000r0000v0009', 10000000.0),
 ('g0000r0000v0006', 10000000.0),
 ('g0000r0000v0005', 10000000.0),
 ('g1240k9360f6750d3990', 10000000.0),
 ('g0000r0000v0000', 10000000.0),
 ('g0000r0000v0008', 10000000.0),
 ('g3680n3760g0500j4810', 10000000.0),
 ('g0000r0000v0003', 10000000.0),
 ('g2470b4690w9360f1130', 10000000.0),
 ('g8300b1150h9280f0000', 10000000.0),
 ('g0000r0000v0007', 10000000.0),
 ('g6090e8020w8380x8390', 5000000.0),
 ('g4150j9680q7200a3410', 1599600.0),
 ('g8090b6110c6310f7000', 1599600.0),
 ('g5720c2850h5040u0380', 1298740.0),
 ('g0640i3610v8090u5600', 1000000.0),
 ('g3370f5680q8510e0660', 1000000.0),
 ('g7340y0720c2120q9810', 1000000.0),
 ('g5100p3970y0790l0590', 1000000.0),
 ('g8240v5350c3860u6090', 1000000.0),
 ('g3360j1130b6180h5160', 1000000.0),
 ('g1111r0000v0004', 1000000.0),
 ('g4750o7520b8710o5530', 1000000.0),
 ('g3780s0870x1460t4120', 1000000.0),
 ('g9850f7590v6430e7150', 1000000.0),
 ('g7690x9200i6010w0210', 1000000.0),
 ('g0770e2740d9780t4630', 1000000.0),
 ('